## Sample YOLO image inference

In [1]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [2]:
from autoAimUtils import render
import keyboard
import win32api
from simple_pid import PID

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True, verbose=False, )
model.to(device)
model.eval()


requirements: scipy>=1.4.1 not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'scipy>=1.4.1' ' returned non-zero exit status 1.
YOLOv5  2022-4-27 torch 2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 64, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      

In [3]:
# image = Image.open('./bus.jpg')
# # Inference
# with torch.no_grad():
#     predictions = model(image, size = 640).pred
# print(predictions)


# render(image, predictions[0].cpu().numpy(), model.names)

## Sample PID for mouse control

In [4]:
# %matplotlib inline

# from IPython import display
# import random, time 
# import matplotlib.pyplot as plt
# from simple_pid import PID


# class GameCursor:
#     """
#     Simple simulation of a cursor on screen
#     """

#     def __init__(self):
#         self.position = [0, 0]
        
#     def update(self, xMouseSpeed, yMouseSpeed, dt):
#         if abs(xMouseSpeed + yMouseSpeed) > 0:
#             self.position[0] += 1 * xMouseSpeed * dt
#             self.position[1] += 1 * yMouseSpeed * dt
#         return self.position



# cursor = GameCursor()
# position = cursor.position
# Kp = 10
# Ki = 0.01
# Kd = 0.01
# max = 250

# xpid = PID(Kp, Ki, Kd, setpoint=0)
# ypid = PID(Kp, Ki, Kd, setpoint=0)
# xpid.output_limits = (-1000, 1000)
# ypid.output_limits = (-1000, 1000)

# start_time = time.time()
# last_time = start_time
   
# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1) 

# while time.time() - start_time <= 5:
#     current_time = time.time()
#     dt = current_time - last_time

#     xMouseSpeed = xpid(position[0])
#     yMouseSpeed = ypid(position[1])
#     position = cursor.update(xMouseSpeed, yMouseSpeed, dt)

#     if (int(current_time) - int(last_time) >= 1):

#         xpid.setpoint = random.randint(0, max)
#         ypid.setpoint = random.randint(0, max)

#     last_time = current_time
    
#     plt.scatter(position[0], position[1], label='measured')
#     plt.scatter(xpid.setpoint, ypid.setpoint, label='target')
#     display.display(plt.gcf())
#     plt.figure().clear()
#     plt.xlabel('time')
#     plt.ylabel('Cursor Position')
#     plt.xlim([-10, max+10])
#     plt.ylim([-10, max+10])
#     display.clear_output(wait=True)

## Time screen capture information

In [ ]:
from PIL import ImageGrab
sct = mss() #instantiate the screenshot application NOTE: this must be run before getScreenInfo, otherwise the reported values will be incorrect

monitor = getScreenInfo(centering)

In [ ]:
%%timeit
screenshot = ImageGrab.grab((monitor['left'], monitor['top'], monitor['left'] + monitor['width'], monitor['top'] + monitor['height']) ) 

47.9 ms ± 2.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
img = np.array(sct.grab(monitor)) #capture the image

21.9 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Bringing it all together

In [5]:
# import the relevant libraries
import numpy as np
import cv2
from mss import mss
from autoAimUtils import getScreenInfo

import time
import win32con

#User parameters
show = True # boolean if the user wants to see the model output
centering = .7 #centering is the decimal percentage of the screen to account for, from the center, from 0 to 1
threshold = .5 # threshold is the lowest confidence bound to be allowed to be considered a detection
weaponsFree = False # Should I be allowed to fire automatically

targets = [0]
# animals = [0, 15, 16, 17, 18, 19, 20, 21, 22, 23]

#set up PID for mouse control
Kp = 8
Ki = 0
Kd = 0

pid_limit = 10000


In [6]:
sct = mss() #instantiate the screenshot application NOTE: this must be run before getScreenInfo, otherwise the reported values will be incorrect

monitor = getScreenInfo(centering)

x_center = monitor['width'] / 2
y_center = monitor['height'] / 2

# Begin the loop of screen inference
while True:

    startFrameTime = time.time() # begin a timer to calculate framerate

    img = np.array(sct.grab(monitor)) #capture the image

    predictions = model(img).pred # perform inference of the image.
    framerate = 1/(time.time() - startFrameTime) # stop the timer and calculate the framerate

    cv2.putText(img, str(framerate), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA) #add framerate to the cv2 image

    # add the aim point to the image
    cv2.circle(img,
        (int(x_center), int(y_center)),
        radius=5,
        color=(0, 255, 0),
        thickness=5
    )


    if len(predictions) > 0: # if there were any predicted targets
        closest = 99999 # set an arbitrarily large value for distance to the center
        for box in predictions[0]: # loop through all predicted targets
            if box[4] > threshold and box[5] in targets: # if the prediction is above the user set confidence threshold
                x = (box[2] + box[0]) / 2 # find the x pixel value for the target
                y = ((box[3] + box[1]) / 2) - ((box[3] - box[1]) / 3) # find the y pixel value for the target (scale up by 1/3 to aim for headshots)
                class_num = int(box[5]) # find the class value for the target

                # add the target to the image
                cv2.circle(img,
                    (int(x), int(y)),
                    radius=10,
                    color=(255, 0, 0),
                    thickness=5
                )
                cv2.rectangle(img,
                            (int(box[0]), int(box[1])),
                            (int(box[2]), int(box[3])),
                            color=(0, 255, 0),
                            thickness=2,
                            )
                # cv2.putText(img,
                #             org=(int(box[0]), int(box[1])), 
                #             text=str(model.names[class_num]), 
                #             fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                #             fontScale=1, 
                #             color=(255,0,0),
                #             thickness=2
                #             )

                # Calculate the distance to the target
                dx_ = int(x.item()-x_center)
                dy_ = int(y.item()-y_center)
                dist = (dx_**2 + dy_**2)**(.5)

                # if this target is closer than the previous closest target, save its infromation as the closest target
                if dist < closest:
                    closest = dist
                    # negate the values to move the right direction
                    dx = -dx_ 
                    dy = -dy_

                    cv2.putText(img,
                                org=(int(box[0]), int(box[1])), 
                                text=str(dx) + " " + str(dy), 
                                fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                                fontScale=1, 
                                color=(255,0,0),
                                thickness=2
                                )  

        # When the user requests, control the mouse
        if keyboard.is_pressed('f1') and closest < 99999:
            #check if continuation of autoAim from last frame

            if not autoAim:
                startTime = time.time() #get the start time
                autoAim=True
                #initiate the PID
                xpid = PID(Kp, Ki, Kd, setpoint=0)
                ypid = PID(Kp, Ki, Kd, setpoint=0)
                xpid.output_limits = (-pid_limit, pid_limit)
                ypid.output_limits = (-pid_limit, pid_limit)
            else:
                lastTime = time.time()
                dt = lastTime - startTime

                win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, # use the windows api to move the mouse
                                        int(xpid(dx)*dt), # this is the x increment to move
                                        int(ypid(dy)*dt), # this is the y incrememnt to move
                                        )
                if dist < 50 and weaponsFree:
                    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0)
                    time.sleep(0.001)
                    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0)
                startTime = lastTime #reset my timer
        else:
            autoAim=False
        
    if show:
        cv2.imshow('Model View', img) # if the user wants to see the model view, rearrange the channels and show the image

        if cv2.waitKey(1) & 0xFF == ord('q'): # The wait insures the image is shown, and the other portion of the command allows the user to stop the program by pressing 'q'
            cv2.destroyAllWindows() # destroy the image window
            break # step out of the loop